In [130]:
from __future__ import annotations

import numpy as np
import polars as pl

FOLDER = "NFL_data/"

In [131]:
# ---------------------------------------------------------------------------
# Utility expressions / helpers
# ---------------------------------------------------------------------------

_DEG2RAD = np.pi / 180.0


def _height_to_cm() -> pl.Expr:
    """Convert height string "6-2" → centimetres as a Polars expression."""
    feet = pl.col("height").str.split("-").list.get(0).cast(pl.Int32())
    inches = pl.col("height").str.split("-").list.get(1).cast(pl.Int32())
    return feet * 30.48 + inches * 2.54


def _weight_to_kg() -> pl.Expr:
    return pl.col("weight") * 0.453_592


def _standard_scale(expr: pl.Expr, mean: float, std: float) -> pl.Expr:
    """Return z‑score‑scaled version of *expr* given pre‑computed mean/std."""
    return (expr - mean) / std


def _minmax_scale(expr: pl.Expr, vmin: float, vmax: float) -> pl.Expr:
    return (expr - vmin) / (vmax - vmin)


def _sin_deg(expr: pl.Expr) -> pl.Expr:
    return (expr * _DEG2RAD).sin()


def _cos_deg(expr: pl.Expr) -> pl.Expr:
    return (expr * _DEG2RAD).cos()


def _normalise_clock() -> pl.Expr:
    """Convert MM:SS string to value in [0,1] (0 → 0 s, 1 → 900 s)."""
    minutes = pl.col("gameClock").str.split(":").list.get(0).cast(pl.Int32())
    seconds = pl.col("gameClock").str.split(":").list.get(1).cast(pl.Int32())
    total = minutes * 60 + seconds
    return (total / 900).alias("gameClock")

def add_yards_to_score(df: pl.DataFrame) -> pl.DataFrame:
    """Add `yardsToScore` based on field position and possession."""
    return df.with_columns(
        pl.when(pl.col("yardlineSide") == pl.col("possessionTeam"))
        .then(100 - pl.col("yardlineNumber"))
        .otherwise(pl.col("yardlineNumber"))
        .alias("yardsToScore")
    )


# 1. Players table


In [132]:
players = (
    pl.read_csv(FOLDER + "players.csv")
    .with_columns(
        [
            _height_to_cm().alias("heightMetric"),
            _weight_to_kg().alias("weightMetric"),
            # Position token using categorical codes
            pl.col("position").cast(pl.Categorical).to_physical().alias("position_token"),
        ]
    )
    # Keep only relevant columns
    .select(
        [
            "nflId",
            "heightMetric",
            "weightMetric",
            "position_token",
        ]
    )
)
players

nflId,heightMetric,weightMetric,position_token
i64,f64,f64,u32
25511,193.04,102.0582,0
29550,193.04,148.778176,1
29851,187.96,102.0582,0
30842,198.12,121.109064,2
33084,193.04,98.429464,0
…,…,…,…
55190,185.42,129.27372,4
55200,198.12,120.655472,10
55239,187.96,136.0776,10


In [133]:
# Standard‑scale the metric columns (mean centred, unit variance)
player_stats = players.select([
    pl.mean("heightMetric").alias("hm_mean"),
    pl.std("heightMetric").alias("hm_std"),
    pl.mean("weightMetric").alias("wm_mean"),
    pl.std("weightMetric").alias("wm_std"),
]).to_dict(as_series=False)

player_stats


{'hm_mean': [188.60959340011783],
 'hm_std': [6.725019912094654],
 'wm_mean': [111.48125973364762],
 'wm_std': [21.43494600472655]}

In [134]:
players = players.with_columns(
    [
        _standard_scale(pl.col("heightMetric"), player_stats["hm_mean"][0], player_stats["hm_std"][0]).alias("heightMetric"),
        _standard_scale(pl.col("weightMetric"), player_stats["wm_mean"][0], player_stats["wm_std"][0]).alias("weightMetric"),
    ]
)

players

nflId,heightMetric,weightMetric,position_token
i64,f64,f64,u32
25511,0.658795,-0.439612,0
29550,0.658795,1.740005,1
29851,-0.096594,-0.439612,0
30842,1.414183,0.449164,2
33084,0.658795,-0.608903,0
…,…,…,…
55190,-0.474288,0.830068,4
55200,1.414183,0.428003,10
55239,-0.096594,1.147488,10


# 2. Plays table

In [135]:
plays_raw = pl.read_csv(
    FOLDER + "plays.csv",
    null_values=["NA"]
)

plays_raw

gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,playNullifiedByPenalty,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment,playClockAtSnap,passResult,passLength,targetX,targetY,playAction,dropbackType,dropbackDistance,passLocationType,timeToThrow,timeInTackleBox,timeToSack,passTippedAtLine,unblockedPressure,qbSpike,qbKneel,qbSneak,rushLocationType,penaltyYards,prePenaltyYardsGained,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
i64,i64,str,i64,i64,i64,str,str,str,i64,str,i64,i64,str,i64,f64,f64,f64,str,str,i64,str,i64,f64,f64,bool,str,f64,str,f64,f64,f64,bool,bool,bool,i64,bool,str,i64,i64,i64,f64,f64,f64,bool,str,str,i64,str,str
2022102302,2655,"""(1:54) (Shotgun) J.Burrow pass…",3,1,10,"""CIN""","""ATL""","""CIN""",21,"""01:54""",35,17,"""N""",31,0.982017,0.017983,0.719313,"""EMPTY""","""3x2""",10,"""C""",6,36.69,16.51,false,"""TRADITIONAL""",2.4,"""INSIDE_BOX""",2.99,2.99,null,false,false,false,0,null,null,null,9,9,0.004634,-0.004634,0.702717,true,null,null,0,"""Cover-3""","""Zone"""
2022091809,3698,"""(2:13) (Shotgun) J.Burrow pass…",4,1,10,"""CIN""","""DAL""","""CIN""",8,"""02:13""",17,17,"""N""",18,0.424356,0.575644,0.607746,"""EMPTY""","""3x2""",9,"""C""",4,20.83,20.49,false,"""TRADITIONAL""",1.14,"""INSIDE_BOX""",1.836,1.836,null,false,false,false,0,null,null,null,4,4,0.002847,-0.002847,-0.240509,true,null,null,0,"""Quarters""","""Zone"""
2022103004,3146,"""(2:00) (Shotgun) D.Mills pass …",4,3,12,"""HOU""","""TEN""","""HOU""",20,"""02:00""",3,17,"""N""",30,0.006291,0.993709,-0.291485,"""SHOTGUN""","""2x2""",12,"""C""",-4,26.02,17.56,false,"""TRADITIONAL""",3.2,"""INSIDE_BOX""",2.236,2.236,null,false,false,false,0,null,null,null,6,6,0.000205,-0.000205,-0.21848,true,null,null,0,"""Quarters""","""Zone"""
2022110610,348,"""(9:28) (Shotgun) P.Mahomes pas…",1,2,10,"""KC""","""TEN""","""TEN""",23,"""09:28""",0,0,"""N""",33,0.884223,0.115777,4.249382,"""SHOTGUN""","""2x2""",11,"""C""",-6,38.95,14.19,false,"""TRADITIONAL""",3.02,"""INSIDE_BOX""",2.202,2.202,null,false,false,false,0,null,null,null,4,4,-0.001308,0.001308,-0.427749,true,null,null,0,"""Quarters""","""Zone"""
2022102700,2799,"""(2:16) (Shotgun) L.Jackson up …",3,2,8,"""BAL""","""TB""","""TB""",27,"""02:16""",10,10,"""N""",37,0.410371,0.589629,3.928413,"""PISTOL""","""3x1""",8,"""""",null,null,null,true,"""DESIGNED_RUN""",2.03,null,null,null,null,null,null,null,0,false,"""INSIDE_LEFT""",null,-1,-1,0.027141,-0.027141,-0.638912,false,"""MAN""","""READ OPTION""",0,"""Cover-1""","""Man"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022110604,1051,"""(12:49) (Shotgun) T.Lawrence p…",2,3,4,"""JAX""","""LV""","""JAX""",31,"""12:49""",0,10,"""N""",79,0.207444,0.792556,0.702276,"""SHOTGUN""","""2x2""",9,"""I""",6,72.69,39.92,false,"""TRADITIONAL""",1.67,"""INSIDE_BOX""",1.86,1.86,null,false,false,false,0,null,null,null,0,0,-0.024741,0.024741,-1.391687,true,null,null,0,"""Cover-2""","""Zone"""
2022103005,3492,"""(12:32) (Shotgun) K.Cousins pa…",4,1,10,"""MIN""","""ARI""","""MIN""",25,"""12:32""",28,26,"""N""",35,0.724565,0.275435,1.600202,"""SHOTGUN""","""2x2""",12,"""I""",20,null,null,false,"""TRADITIONAL""",2.24,"""INSIDE_BOX""",3.136,3.136,null,false,false,false,0,null,null,null,0,0,-0.02658,0.02658,-0.503208,true,null,null,0,"""Cover-3""","""Zone"""
2022092502,2337,"""(9:59) (Shotgun) P.Mahomes scr…",3,1,10,"""KC""","""IND""","""IND""",13,"""09:59""",10,14,"""N""",97,0.165774,0.834226,5.127508,"""SHOTGUN""","""2x2""",6,"""R""",null,null,null,false,"""SCRAMBLE""",3.28,null,null,null,null,null,false,null,0,false,"""INSIDE_RIGHT""",null,10,10,-0.01379,0.01379,1.073898,true,null,null,0,""

In [136]:
plays = (
    plays_raw.select(
        [
            "gameId",
            "playId",
            "quarter",
            "down",
            "yardsToGo",
            "possessionTeam",
            "defensiveTeam",
            "yardlineSide",
            "yardlineNumber",
            "absoluteYardlineNumber",
            "gameClock",
            "preSnapHomeScore",
            "preSnapVisitorScore",
        ]
    )
    .to_dummies(columns=["quarter", "down"], drop_first=True)
    .with_columns(
        [
            _normalise_clock(),
            (pl.col("preSnapHomeScore") / 50).alias("preSnapHomeScore"),
            (pl.col("preSnapVisitorScore") / 50).alias("preSnapVisitorScore"),
        ]
    )
)
plays

gameId,playId,quarter_1,quarter_2,quarter_4,quarter_5,down_2,down_3,down_4,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,absoluteYardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore
i64,i64,u8,u8,u8,u8,u8,u8,u8,i64,str,str,str,i64,i64,f64,f64,f64
2022102302,2655,0,0,0,0,0,0,0,10,"""CIN""","""ATL""","""CIN""",21,31,0.126667,0.7,0.34
2022091809,3698,0,0,1,0,0,0,0,10,"""CIN""","""DAL""","""CIN""",8,18,0.147778,0.34,0.34
2022103004,3146,0,0,1,0,0,1,0,12,"""HOU""","""TEN""","""HOU""",20,30,0.133333,0.06,0.34
2022110610,348,1,0,0,0,1,0,0,10,"""KC""","""TEN""","""TEN""",23,33,0.631111,0.0,0.0
2022102700,2799,0,0,0,0,1,0,0,8,"""BAL""","""TB""","""TB""",27,37,0.151111,0.2,0.2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022110604,1051,0,1,0,0,0,1,0,4,"""JAX""","""LV""","""JAX""",31,79,0.854444,0.0,0.2
2022103005,3492,0,0,1,0,0,0,0,10,"""MIN""","""ARI""","""MIN""",25,35,0.835556,0.56,0.52
2022092502,2337,0,0,0,0,0,0,0,10,"""KC""","""IND""","""IND""",13,97,0.665556,0.2,0.28


In [137]:
# derive yardsToScore
plays = add_yards_to_score(plays)

plays.select(['yardlineSide', 'yardsToScore', 'possessionTeam', 'defensiveTeam','yardlineNumber'])

yardlineSide,yardsToScore,possessionTeam,defensiveTeam,yardlineNumber
str,i64,str,str,i64
"""CIN""",79,"""CIN""","""ATL""",21
"""CIN""",92,"""CIN""","""DAL""",8
"""HOU""",80,"""HOU""","""TEN""",20
"""TEN""",23,"""KC""","""TEN""",23
"""TB""",27,"""BAL""","""TB""",27
…,…,…,…,…
"""JAX""",69,"""JAX""","""LV""",31
"""MIN""",75,"""MIN""","""ARI""",25
"""IND""",13,"""KC""","""IND""",13


In [138]:
# drop IDs no longer needed for modelling
plays = plays.drop([
    "possessionTeam",
    "defensiveTeam",
    "yardlineSide",
    "yardlineNumber",
    "absoluteYardlineNumber"
])

plays

gameId,playId,quarter_1,quarter_2,quarter_4,quarter_5,down_2,down_3,down_4,yardsToGo,gameClock,preSnapHomeScore,preSnapVisitorScore,yardsToScore
i64,i64,u8,u8,u8,u8,u8,u8,u8,i64,f64,f64,f64,i64
2022102302,2655,0,0,0,0,0,0,0,10,0.126667,0.7,0.34,79
2022091809,3698,0,0,1,0,0,0,0,10,0.147778,0.34,0.34,92
2022103004,3146,0,0,1,0,0,1,0,12,0.133333,0.06,0.34,80
2022110610,348,1,0,0,0,1,0,0,10,0.631111,0.0,0.0,23
2022102700,2799,0,0,0,0,1,0,0,8,0.151111,0.2,0.2,27
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022110604,1051,0,1,0,0,0,1,0,4,0.854444,0.0,0.2,69
2022103005,3492,0,0,1,0,0,0,0,10,0.835556,0.56,0.52,75
2022092502,2337,0,0,0,0,0,0,0,10,0.665556,0.2,0.28,13


# 3. Tracking data (single week)

In [139]:
tracking_raw = pl.read_csv(
    FOLDER + "tracking_week_1.csv",
    null_values="NA",                    # treat "NA" as null
    infer_schema_length=10000,           # safer for wide CSVs
    dtypes={"nflId": pl.Int64},          # load as integer with nulls allowed
)

tracking_raw

C:\Users\sebas\AppData\Local\Temp\ipykernel_39232\929280133.py:1: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  tracking_raw = pl.read_csv(


gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
i64,i64,i64,str,i64,str,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,str
2022091200,64,35459,"""Kareem Jackson""",1,"""BEFORE_SNAP""","""2022-09-13 00:16:03.5""",22,"""DEN""","""right""",51.06,28.55,0.72,0.37,0.07,246.17,68.34,"""huddle_break_offense"""
2022091200,64,35459,"""Kareem Jackson""",2,"""BEFORE_SNAP""","""2022-09-13 00:16:03.6""",22,"""DEN""","""right""",51.13,28.57,0.71,0.36,0.07,245.41,71.21,null
2022091200,64,35459,"""Kareem Jackson""",3,"""BEFORE_SNAP""","""2022-09-13 00:16:03.7""",22,"""DEN""","""right""",51.2,28.59,0.69,0.23,0.07,244.45,69.9,null
2022091200,64,35459,"""Kareem Jackson""",4,"""BEFORE_SNAP""","""2022-09-13 00:16:03.8""",22,"""DEN""","""right""",51.26,28.62,0.67,0.22,0.07,244.45,67.98,null
2022091200,64,35459,"""Kareem Jackson""",5,"""BEFORE_SNAP""","""2022-09-13 00:16:03.9""",22,"""DEN""","""right""",51.32,28.65,0.65,0.34,0.07,245.74,62.83,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022090800,3696,null,"""football""",171,"""AFTER_SNAP""","""2022-09-09 03:07:37.7""",null,"""football""","""left""",1.7,0.62,14.42,5.36,1.47,null,null,"""pass_outcome_incomplete"""
2022090800,3696,null,"""football""",172,"""AFTER_SNAP""","""2022-09-09 03:07:37.8""",null,"""football""","""left""",0.59,0.01,3.6,0.06,1.27,null,null,null
2022090800,3696,null,"""football""",173,"""AFTER_SNAP""","""2022-09-09 03:07:37.9""",null,"""football""","""left""",0.23,-0.07,3.6,0.23,0.38,null,null,null


In [140]:
tracking = (
    tracking_raw.with_columns(
        [
            _sin_deg(pl.col("o")).alias("o_sin"),
            _cos_deg(pl.col("o")).alias("o_cos"),
            _sin_deg(pl.col("dir")).alias("dir_sin"),
            _cos_deg(pl.col("dir")).alias("dir_cos"),
        ]
    )
    .drop([
        "displayName",
        "jerseyNumber",
        "time",
        "frameType",
        "playDirection",
        "o",
        "dir",
        "club",
    ])
)

tracking

gameId,playId,nflId,frameId,x,y,s,a,dis,event,o_sin,o_cos,dir_sin,dir_cos
i64,i64,i64,i64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64
2022091200,64,35459,1,51.06,28.55,0.72,0.37,0.07,"""huddle_break_offense""",-0.914748,-0.404024,0.92939,0.369098
2022091200,64,35459,2,51.13,28.57,0.71,0.36,0.07,null,-0.909309,-0.416122,0.946705,0.3221
2022091200,64,35459,3,51.2,28.59,0.69,0.23,0.07,null,-0.902209,-0.431299,0.939094,0.34366
2022091200,64,35459,4,51.26,28.62,0.67,0.22,0.07,null,-0.902209,-0.431299,0.927053,0.37493
2022091200,64,35459,5,51.32,28.65,0.65,0.34,0.07,null,-0.91169,-0.410878,0.889656,0.456632
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022090800,3696,null,171,1.7,0.62,14.42,5.36,1.47,"""pass_outcome_incomplete""",null,null,null,null
2022090800,3696,null,172,0.59,0.01,3.6,0.06,1.27,null,null,null,null,null
2022090800,3696,null,173,0.23,-0.07,3.6,0.23,0.38,null,null,null,null,null


In [141]:
SCALE_COLS = ["x", "y", "s", "a"]
mins = tracking.select([pl.min(c).alias(f"{c}_min") for c in SCALE_COLS]).to_dict(as_series=False)
maxs = tracking.select([pl.max(c).alias(f"{c}_max") for c in SCALE_COLS]).to_dict(as_series=False)

for col in SCALE_COLS:
    tracking = tracking.with_columns(
        _minmax_scale(pl.col(col), mins[f"{col}_min"][0], maxs[f"{col}_max"][0]).alias(col)
    )

tracking

gameId,playId,nflId,frameId,x,y,s,a,dis,event,o_sin,o_cos,dir_sin,dir_cos
i64,i64,i64,i64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64
2022091200,64,35459,1,0.429512,0.478128,0.024708,0.006539,0.07,"""huddle_break_offense""",-0.914748,-0.404024,0.92939,0.369098
2022091200,64,35459,2,0.430047,0.478383,0.024365,0.006363,0.07,null,-0.909309,-0.416122,0.946705,0.3221
2022091200,64,35459,3,0.430583,0.478638,0.023679,0.004065,0.07,null,-0.902209,-0.431299,0.939094,0.34366
2022091200,64,35459,4,0.431042,0.479021,0.022992,0.003888,0.07,null,-0.902209,-0.431299,0.927053,0.37493
2022091200,64,35459,5,0.431502,0.479403,0.022306,0.006009,0.07,null,-0.91169,-0.410878,0.889656,0.456632
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022090800,3696,null,171,0.051737,0.121923,0.494852,0.094733,1.47,"""pass_outcome_incomplete""",null,null,null,null
2022090800,3696,null,172,0.043242,0.114144,0.123542,0.00106,1.27,null,null,null,null,null
2022090800,3696,null,173,0.040487,0.113123,0.123542,0.004065,0.38,null,null,null,null,null


In [142]:
event_df = (
    tracking.select(["gameId", "playId", "frameId", "event"])
    .with_columns(pl.col("event").fill_null("Nothing"))
    .to_dummies(columns=["event"], drop_first=True)  # `prefix` arg removed in Polars >=0.20
    .unique()
)
tracking = tracking.drop("event")

event_df

gameId,playId,frameId,event_Nothing,event_ball_snap,event_dropped_pass,event_first_contact,event_fumble,event_fumble_defense_recovered,event_fumble_offense_recovered,event_handoff,event_huddle_start_offense,event_lateral,event_line_set,event_man_in_motion,event_out_of_bounds,event_pass_arrived,event_pass_forward,event_pass_outcome_caught,event_pass_outcome_incomplete,event_pass_outcome_interception,event_pass_outcome_touchdown,event_pass_shovel,event_pass_tipped,event_play_action,event_play_submit,event_qb_kneel,event_qb_sack,event_qb_slide,event_qb_spike,event_qb_strip_sack,event_run,event_run_pass_option,event_shift,event_snap_direct,event_tackle,event_timeout_away,event_touchback,event_touchdown
i64,i64,i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
2022091106,236,18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022091110,2949,99,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022091106,950,53,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022091107,3026,67,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022091106,2215,18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022091104,3369,81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022091106,523,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022091104,3204,259,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 4. Flatten 23‑entity (22 players + ball) frames

In [143]:
VALUE_COLS = [
    "x", "y", "s", "a", "dis", "o_sin", "o_cos", "dir_sin", "dir_cos",
]

complete_frames = (
    tracking.group_by(["gameId", "playId", "frameId"]).count()
    .filter(pl.col("count") == 23)
    .select(["gameId", "playId", "frameId"])
)
tracking_complete = tracking.join(complete_frames, on=["gameId", "playId", "frameId"], how="inner")

tracking_ordered = (
    tracking_complete
    .sort(["gameId", "playId", "frameId", "nflId"], nulls_last=True)  # ball last
    .with_columns(
        pl.int_range(1, 24).over(["gameId", "playId", "frameId"]).alias("entity_order")
    )
)

agg_exprs = [pl.col(c).sort_by("entity_order").alias(c) for c in VALUE_COLS]
flat_lists = tracking_ordered.group_by(["gameId", "playId", "frameId"]).agg(agg_exprs)

flat_wide = flat_lists.select(["gameId", "playId", "frameId"])
for m in VALUE_COLS:
    flat_wide = flat_wide.join(
        flat_lists.select(
            ["gameId", "playId", "frameId"]
            + [pl.col(m).list.get(i).alias(f"{m}_{i+1}") for i in range(23)]
        ),
        on=["gameId", "playId", "frameId"],
        how="left",
    )

flat_wide

C:\Users\sebas\AppData\Local\Temp\ipykernel_39232\3717328703.py:6: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  tracking.group_by(["gameId", "playId", "frameId"]).count()


gameId,playId,frameId,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,x_20,x_21,x_22,x_23,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,…,dir_sin_10,dir_sin_11,dir_sin_12,dir_sin_13,dir_sin_14,dir_sin_15,dir_sin_16,dir_sin_17,dir_sin_18,dir_sin_19,dir_sin_20,dir_sin_21,dir_sin_22,dir_sin_23,dir_cos_1,dir_cos_2,dir_cos_3,dir_cos_4,dir_cos_5,dir_cos_6,dir_cos_7,dir_cos_8,dir_cos_9,dir_cos_10,dir_cos_11,dir_cos_12,dir_cos_13,dir_cos_14,dir_cos_15,dir_cos_16,dir_cos_17,dir_cos_18,dir_cos_19,dir_cos_20,dir_cos_21,dir_cos_22,dir_cos_23
i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2022091101,2320,90,0.302924,0.302847,0.344788,0.298867,0.3167,0.306291,0.308511,0.383055,0.250344,0.323664,0.302541,0.346013,0.319225,0.313485,0.305679,0.317848,0.390862,0.301546,0.338359,0.353896,0.301776,0.296265,0.314098,0.440377,0.259661,0.437699,0.416401,0.401734,0.482336,0.414871,0.336437,0.415253,0.665221,0.392297,…,0.297875,0.999512,-0.969999,-0.997146,-0.840377,-0.146256,-0.994774,-0.433345,0.925871,0.72369,0.821945,0.565687,-0.862602,null,-0.013613,-0.107132,-0.452435,0.338574,-0.868977,-0.371044,-0.643589,-0.536269,-0.755396,-0.954605,-0.031236,0.243107,0.075501,-0.542002,0.989247,-0.102098,-0.901228,0.377841,-0.690125,-0.569566,-0.82462,0.505883,null
2022091102,1562,34,0.733736,0.703352,0.760294,0.725241,0.669371,0.710929,0.703582,0.734272,0.731747,0.756161,0.706184,0.743916,0.689499,0.70649,0.670442,0.743916,0.760677,0.731823,0.676182,0.651615,0.731976,0.708557,0.721338,0.462951,0.410279,0.53131,0.494707,0.453259,0.480296,0.505803,0.538324,0.422268,0.368065,0.456447,…,-0.286525,0.837814,-0.37315,-0.46978,0.711045,0.692017,-0.481754,0.814217,-0.897874,0.204325,0.031411,-0.832535,-0.281337,null,0.159192,-0.992418,0.561361,-0.988415,-0.987331,-0.241753,0.473473,0.685691,-0.782282,-0.958073,0.545956,0.927771,-0.882784,0.703147,0.721881,-0.876307,0.580561,-0.440253,-0.978903,-0.999507,-0.553973,-0.959609,null
2022091200,3245,81,0.248355,0.246518,0.231058,0.156666,0.245064,0.204653,0.242308,0.23343,0.244681,0.180928,0.144803,0.247206,0.239783,0.243686,0.205266,0.203352,0.211006,0.232971,0.239783,0.247283,0.294887,0.234578,0.245446,0.495473,0.535646,0.512052,0.425583,0.444331,0.403265,0.516133,0.470221,0.462313,0.624155,0.54011,…,-0.857527,-0.724533,-0.937282,-0.978293,-0.179489,0.967135,0.986629,0.69704,0.697415,-0.18378,-0.254939,-0.475778,0.897566,null,0.348408,0.967533,0.694407,0.87224,0.995979,0.985259,0.863484,0.892822,0.927576,-0.51444,0.68924,0.348572,-0.207229,0.98376,-0.254264,-0.162982,0.717032,0.716667,0.982967,0.966957,0.879566,-0.440879,null
2022091101,1537,131,0.456452,0.458978,0.498163,0.471835,0.432879,0.477193,0.495331,0.459207,0.465177,0.42844,0.577683,0.409613,0.458289,0.473519,0.50176,0.45959,0.480254,0.445431,0.469998,0.476121,0.469692,0.456758,0.466095,0.512945,0.614845,0.485907,0.42367,0.491646,0.455809,0.516133,0.434511,0.496238,0.452111,0.49611,…,-0.656717,0.994245,0.385584,0.621968,-0.659739,0.22376,-0.60446,-0.534942,0.077415,0.335452,0.983064,0.344971,0.947768,null,0.738102,-0.82085,-0.890372,-0.390731,0.81533,-0.234633,0.099146,0.739514,-0.874366,-0.754137,0.107132,-0.922673,0.783043,-0.751494,0.974644,0.796635,0.844889,0.996999,-0.942057,-0.183265,0.938613,-0.318959,null
2022091200,4012,45,0.580361,0.517067,0.449181,0.533599,0.521812,0.577606,0.555411,0.565514,0.524338,0.532451,0.539874,0.52097,0.537043,0.521965,0.508878,0.522425,0.509949,0.564978,0.547451,0.535206,0.521506,0.521353,0.525563,0.453769,0.419079,0.418186,0.399184,0.4025,0.374825,0.422778,0.370616,0.419334,0.418569,0.462824,…,-0.835328,-0.49879,0.206375,-0.972411,-0.62932,-0.994207,0.63527,0.734086,0.008901,-0.99949,-0.336274,0.561939,-0.796319,null,0.479764,0.019895,-0.98

In [144]:
model_input = (
    flat_wide
    .join(event_df, on=["gameId", "playId", "frameId"], how="left")
    .join(plays, on=["gameId", "playId"], how="left")
)

model_input.write_parquet("model_input.parquet")
print(f"✓ Pipeline finished — wrote model_input.parquet ({len(model_input)} rows)")

✓ Pipeline finished — wrote model_input.parquet (308900 rows)


In [145]:
model_input.columns

['gameId',
 'playId',
 'frameId',
 'x_1',
 'x_2',
 'x_3',
 'x_4',
 'x_5',
 'x_6',
 'x_7',
 'x_8',
 'x_9',
 'x_10',
 'x_11',
 'x_12',
 'x_13',
 'x_14',
 'x_15',
 'x_16',
 'x_17',
 'x_18',
 'x_19',
 'x_20',
 'x_21',
 'x_22',
 'x_23',
 'y_1',
 'y_2',
 'y_3',
 'y_4',
 'y_5',
 'y_6',
 'y_7',
 'y_8',
 'y_9',
 'y_10',
 'y_11',
 'y_12',
 'y_13',
 'y_14',
 'y_15',
 'y_16',
 'y_17',
 'y_18',
 'y_19',
 'y_20',
 'y_21',
 'y_22',
 'y_23',
 's_1',
 's_2',
 's_3',
 's_4',
 's_5',
 's_6',
 's_7',
 's_8',
 's_9',
 's_10',
 's_11',
 's_12',
 's_13',
 's_14',
 's_15',
 's_16',
 's_17',
 's_18',
 's_19',
 's_20',
 's_21',
 's_22',
 's_23',
 'a_1',
 'a_2',
 'a_3',
 'a_4',
 'a_5',
 'a_6',
 'a_7',
 'a_8',
 'a_9',
 'a_10',
 'a_11',
 'a_12',
 'a_13',
 'a_14',
 'a_15',
 'a_16',
 'a_17',
 'a_18',
 'a_19',
 'a_20',
 'a_21',
 'a_22',
 'a_23',
 'dis_1',
 'dis_2',
 'dis_3',
 'dis_4',
 'dis_5',
 'dis_6',
 'dis_7',
 'dis_8',
 'dis_9',
 'dis_10',
 'dis_11',
 'dis_12',
 'dis_13',
 'dis_14',
 'dis_15',
 'dis_16',
 'dis_1

# Loading full file with all 9 tracking data files

## Polars

In [2]:
import polars as pl

# Load parquet
df_polars = pl.read_parquet("model_input.parquet")

# Show the first few rows
df_polars.head()


gameId,playId,frameId,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,x_20,x_21,x_22,x_23,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,…,event_pass_forward,event_pass_outcome_caught,event_pass_outcome_incomplete,event_pass_outcome_interception,event_pass_outcome_touchdown,event_pass_shovel,event_pass_tipped,event_penalty_accepted,event_penalty_flag,event_play_action,event_play_submit,event_qb_kneel,event_qb_sack,event_qb_slide,event_qb_spike,event_qb_strip_sack,event_run,event_run_pass_option,event_safety,event_shift,event_snap_direct,event_tackle,event_timeout_away,event_touchback,event_touchdown,quarter_1,quarter_2,quarter_4,quarter_5,down_2,down_3,down_4,yardsToGo,gameClock,preSnapHomeScore,preSnapVisitorScore,yardsToScore
i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,f64,f64,f64,i64
2022100908,2462,80,0.627241,0.545798,0.590756,0.603431,0.59972,0.585224,0.491317,0.621148,0.543768,0.589916,0.598319,0.589006,0.552941,0.603011,0.544398,0.493417,0.595098,0.588165,0.590616,0.595168,0.55056,0.585924,0.618908,0.530744,0.295671,0.300444,0.49212,0.55394,0.529745,0.613652,0.504883,0.51687,0.471143,0.690122,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0.633333,0.26,0.0,62
2022101603,3746,24,0.241247,0.201681,0.269048,0.268557,0.246989,0.282703,0.27423,0.285504,0.25098,0.235574,0.282283,0.228641,0.270798,0.246569,0.223529,0.233403,0.276751,0.313726,0.252521,0.276541,0.263726,0.188655,0.259314,0.411654,0.434739,0.450721,0.483796,0.461154,0.421865,0.434517,0.536848,0.466371,0.521643,0.514095,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,10,0.28,0.52,0.42,15
2022103011,1235,21,0.670798,0.67542,0.667857,0.695098,0.694328,0.654062,0.736555,0.672129,0.667437,0.657493,0.719608,0.714846,0.661695,0.664356,0.712115,0.714426,0.702031,0.667087,0.697409,0.713025,0.695098,0.669608,0.680882,0.534628,0.523085,0.520311,0.534628,0.550499,0.516648,0.537181,0.559157,0.531743,0.545172,0.475916,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,10,0.807778,0.14,0.0,75
2022091807,1030,122,0.276751,0.147829,0.192297,0.172619,0.190476,0.19916,0.191387,0.179132,0.228571,0.226681,0.20007,0.17612,0.195938,0.183473,0.20035,0.183754,0.168067,0.194398,0.187675,0.203361,0.186555,0.188866,0.17388,0.536737,0.477359,0.479689,0.510433,0.555383,0.557492,0.585905,0.563374,0.470477,0.452053,0.537958,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,8,0.637778,0.14,0.0,8
2022100901,2491,46,0.668838,0.64958,0.665756,0.67612,0.627661,0.667017,0.65077,0.652381,0.696849,0.569538,0.652871,0.636905,0.619818,0.69972,0.652101,0.669468,0.64923,0.666527,0.670098,0.582633,0.67577,0.667927,0.660014,0.519645,0.52586,0.536848,0.322198,0.576027,0.498224,0.553274,0.413319,0.536071,0.461931,0.317425,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0.687778,0.62,0.06,72


In [3]:
df_polars.columns

['gameId',
 'playId',
 'frameId',
 'x_1',
 'x_2',
 'x_3',
 'x_4',
 'x_5',
 'x_6',
 'x_7',
 'x_8',
 'x_9',
 'x_10',
 'x_11',
 'x_12',
 'x_13',
 'x_14',
 'x_15',
 'x_16',
 'x_17',
 'x_18',
 'x_19',
 'x_20',
 'x_21',
 'x_22',
 'x_23',
 'y_1',
 'y_2',
 'y_3',
 'y_4',
 'y_5',
 'y_6',
 'y_7',
 'y_8',
 'y_9',
 'y_10',
 'y_11',
 'y_12',
 'y_13',
 'y_14',
 'y_15',
 'y_16',
 'y_17',
 'y_18',
 'y_19',
 'y_20',
 'y_21',
 'y_22',
 'y_23',
 's_1',
 's_2',
 's_3',
 's_4',
 's_5',
 's_6',
 's_7',
 's_8',
 's_9',
 's_10',
 's_11',
 's_12',
 's_13',
 's_14',
 's_15',
 's_16',
 's_17',
 's_18',
 's_19',
 's_20',
 's_21',
 's_22',
 's_23',
 'a_1',
 'a_2',
 'a_3',
 'a_4',
 'a_5',
 'a_6',
 'a_7',
 'a_8',
 'a_9',
 'a_10',
 'a_11',
 'a_12',
 'a_13',
 'a_14',
 'a_15',
 'a_16',
 'a_17',
 'a_18',
 'a_19',
 'a_20',
 'a_21',
 'a_22',
 'a_23',
 'dis_1',
 'dis_2',
 'dis_3',
 'dis_4',
 'dis_5',
 'dis_6',
 'dis_7',
 'dis_8',
 'dis_9',
 'dis_10',
 'dis_11',
 'dis_12',
 'dis_13',
 'dis_14',
 'dis_15',
 'dis_16',
 'dis_1

In [ ]:
df_polars.null_count()

gameId,playId,frameId,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,x_20,x_21,x_22,x_23,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,…,event_pass_forward,event_pass_outcome_caught,event_pass_outcome_incomplete,event_pass_outcome_interception,event_pass_outcome_touchdown,event_pass_shovel,event_pass_tipped,event_penalty_accepted,event_penalty_flag,event_play_action,event_play_submit,event_qb_kneel,event_qb_sack,event_qb_slide,event_qb_spike,event_qb_strip_sack,event_run,event_run_pass_option,event_safety,event_shift,event_snap_direct,event_tackle,event_timeout_away,event_touchback,event_touchdown,quarter_1,quarter_2,quarter_4,quarter_5,down_2,down_3,down_4,yardsToGo,gameClock,preSnapHomeScore,preSnapVisitorScore,yardsToScore
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Pandas

In [148]:
! pip install pyarrow

In [149]:
import pandas as pd

# Load parquet
df_pandas = pd.read_parquet("model_input.parquet")

# View data
df_pandas.head()


,gameId,playId,frameId,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,quarter_4,quarter_5,down_2,down_3,down_4,yardsToGo,gameClock,preSnapHomeScore,preSnapVisitorScore,yardsToScore
0,2022091101,2320,90,0.302924,0.302847,0.344788,0.298867,0.316700,0.306291,0.308511,...,0,0,1,0,0,10,0.991111,0.14,0.34,74
1,2022091102,1562,34,0.733736,0.703352,0.760294,0.725241,0.669371,0.710929,0.703582,...,0,0,0,1,0,13,0.468889,0.00,0.14,79
2,2022091200,3245,81,0.248355,0.246518,0.231058,0.156666,0.245064,0.204653,0.242308,...,1,0,0,0,0,10,0.526667,0.34,0.26,16
3,2022091101,1537,131,0.456452,0.458978,0.498163,0.471835,0.432879,0.477193,0.495331,...,0,0,1,0,0,2,0.418889,0.00,0.14,54
4,2022091200,4012,45,0.580361,0.517067,0.449181,0.533599,0.521812,0.577606,0.555411,...,1,0,0,0,0,10,0.016667,0.34,0.32,46


In [150]:
df_pandas.columns


Index(['gameId', 'playId', 'frameId', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6',
       'x_7',
       ...
       'quarter_4', 'quarter_5', 'down_2', 'down_3', 'down_4', 'yardsToGo',
       'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'yardsToScore'],
      dtype='object', length=258)